# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
%pip install yfinance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.4 MB/s eta 0:00:0000:0100:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=533cd0760dd1404109f177cd104deba19fb36eb333cff55af104097bda4abfef
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-linux_x86_64.whl size=1049307 sha256=c202cb76e2e6a40270d10f76a8105753b7a95eb87fdfa9e2cdd8d22c7821e570
  Stored in directory: /home/codespace/.cache/pip/wheels/d1/df/a9/0202b051c65b11c992dd6db9f2babdd2c44ec7d35d511be5d3


In [22]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

BLACKBOARD_RELEASE_DATE = "2025-09-01"
LAST_NAME_INITIAL = "C"

required = ["AAPL", "SPY", "TSLA"]
starts_with_initial = ["COST"]
others = ["MSFT", "NVDA"]

tickers = required + starts_with_initial + others
tickers = list(dict.fromkeys([t.upper() for t in tickers]))

print("Universe:", tickers)
assert all(t in tickers for t in required), "Must include AAPL, SPY, TSLA."
assert any(t.upper().startswith(LAST_NAME_INITIAL.upper()) for t in tickers), \
       f"At least one ticker must start with '{LAST_NAME_INITIAL}'."

release = pd.to_datetime(BLACKBOARD_RELEASE_DATE)
last_included_month_end = release - pd.offsets.MonthEnd(1)
month_ends = pd.date_range(end=last_included_month_end, periods=24, freq="ME")

print("Last included month-end:", last_included_month_end.date())
print(f"Target 24 months: {month_ends.min().date()} → {month_ends.max().date()} (24 month-ends)")

start_download = (month_ends.min() - pd.offsets.Day(7)).date()
end_download   = (month_ends.max() + pd.offsets.Day(7)).date()
raw = yf.download(tickers, start=start_download, end=end_download, auto_adjust=False, progress=False)

px = raw["Adj Close"].copy()
px_m_all = px.resample("ME").last()
px_m = px_m_all.reindex(month_ends).ffill()

missing_by_ticker = px_m.isna().sum().sort_values(ascending=False)
if missing_by_ticker.any():
    print("Missing data counts over 24 months:")
    display(missing_by_ticker.to_frame("N_missing"))
    bad = list(missing_by_ticker[missing_by_ticker > 0].index)
    raise ValueError(
        f"Some tickers lack 24 months of data in the required window: {bad}. "
        "Swap tickers (keep AAPL, SPY, TSLA + one starting with your initial) or verify the release date."
    )

assert px_m.shape == (24, len(tickers)), f"Expected 24×{len(tickers)}, got {px_m.shape}"

print("Collected 24 consecutive month-end prices (Adj Close):")
display(px_m.tail(6))

Universe: ['AAPL', 'SPY', 'TSLA', 'COST', 'MSFT', 'NVDA']
Last included month-end: 2025-08-31
Target 24 months: 2023-09-30 → 2025-08-31 (24 month-ends)
Collected 24 consecutive month-end prices (Adj Close):


Ticker,AAPL,COST,MSFT,NVDA,SPY,TSLA
2025-03-31,221.587616,943.242554,374.087158,108.372475,557.741150,259.160004
2025-04-30,211.981125,991.831848,393.888184,108.912437,552.905457,282.160004
2025-05-31,200.622314,1038.740967,459.604431,135.120621,587.652771,346.459991
2025-06-30,204.937408,988.570435,496.593658,157.990005,617.849976,317.660004
2025-07-31,207.334702,938.340027,532.624390,177.869995,632.080017,308.269989
2025-08-31,232.139999,943.320007,506.690002,174.179993,645.049988,333.869995


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [23]:
# YOUR CHANGES HERE

out = px_m.copy().reset_index()
out = out.rename(columns={out.columns[0]: "date"})
out["date"] = pd.to_datetime(out["date"]).dt.strftime("%Y-%m-%d")

out.to_csv("historical_prices.tsv", sep="\t", index=False)

print("Saved historical_prices.tsv with shape:", out.shape)
display(out.head())

Saved historical_prices.tsv with shape: (24, 7)


Ticker,date,AAPL,COST,MSFT,NVDA,SPY,TSLA
0,2023-09-30,169.549271,546.110657,311.062317,43.475830,417.865662,250.220001
1,2023-10-31,169.113556,534.008423,333.090363,40.758274,408.794312,200.839996
2,2023-11-30,188.355316,574.015381,374.042206,46.745083,446.135193,240.080002
3,2023-12-31,190.913666,653.764404,371.209167,49.499973,466.503693,248.479996
4,2024-01-31,182.851913,688.231384,392.472412,61.499638,473.933411,187.289993


In [24]:
last_included_bme = release - pd.offsets.BusinessMonthEnd(1)
bm_ends = pd.date_range(end=last_included_bme, periods=24, freq="BM")

px_b = px.asfreq("B").ffill()
px_m = px_b.reindex(bm_ends)

assert px_m.shape == (24, len(tickers)), f"Expected 24×{len(tickers)}, got {px_m.shape}"
assert not px_m.isna().any().any(), "Missing data after BME alignment."

print("Rebuilt px_m with Business Month End dates:")
display(px_m.head(3))
display(px_m.tail(3))

Rebuilt px_m with Business Month End dates:


/tmp/ipykernel_13363/2207026134.py:2: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  bm_ends = pd.date_range(end=last_included_bme, periods=24, freq="BM")


Ticker,AAPL,COST,MSFT,NVDA,SPY,TSLA
2023-09-29,169.549271,546.110657,311.062317,43.475830,417.865662,250.220001
2023-10-31,169.113556,534.008423,333.090363,40.758274,408.794312,200.839996
2023-11-30,188.355316,574.015381,374.042206,46.745083,446.135193,240.080002


Ticker,AAPL,COST,MSFT,NVDA,SPY,TSLA
2025-06-30,204.937408,988.570435,496.593658,157.990005,617.849976,317.660004
2025-07-31,207.334702,938.340027,532.624390,177.869995,632.080017,308.269989
2025-08-29,232.139999,943.320007,506.690002,174.179993,645.049988,333.869995


Submit "historical_prices.tsv" in Gradescope.

In [25]:
last_included_bme = release - pd.offsets.BusinessMonthEnd(1)
bm_ends = pd.date_range(end=last_included_bme, periods=24, freq="BME")

px_b = px.asfreq("B").ffill()
px_m = px_b.reindex(bm_ends)

assert px_m.shape == (24, len(tickers))
assert not px_m.isna().any().any()

out = px_m.reset_index()
out = out.rename(columns={out.columns[0]: "date"})
out["date"] = pd.to_datetime(out["date"]).dt.strftime("%Y-%m-%d")
out.to_csv("historical_prices.tsv", sep="\t", index=False)

print("Saved historical_prices.tsv with shape:", out.shape)
display(out.head())

Saved historical_prices.tsv with shape: (24, 7)


Ticker,date,AAPL,COST,MSFT,NVDA,SPY,TSLA
0,2023-09-29,169.549271,546.110657,311.062317,43.475830,417.865662,250.220001
1,2023-10-31,169.113556,534.008423,333.090363,40.758274,408.794312,200.839996
2,2023-11-30,188.355316,574.015381,374.042206,46.745083,446.135193,240.080002
3,2023-12-29,190.913666,653.764404,371.209167,49.499973,466.503693,248.479996
4,2024-01-31,182.851913,688.231384,392.472412,61.499638,473.933411,187.289993


## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [26]:
# YOUR CHANGES HERE

prices = pd.read_csv("historical_prices.tsv", sep="\t")
prices["date"] = pd.to_datetime(prices["date"])
prices = prices.set_index("date").sort_index()

rets_m = prices.pct_change().dropna()

log_rets_m = np.log(prices).diff().dropna()

out = rets_m.copy().reset_index()
out["date"] = out["date"].dt.strftime("%Y-%m-%d")
out.to_csv("asset_returns.tsv", sep="\t", index=False)

print("Saved asset_returns.tsv with shape:", out.shape)

assert rets_m.shape[0] == 23, f"Expected 23 monthly return rows, got {rets_m.shape[0]}"
assert not rets_m.isna().any().any(), "NaNs found in returns."

print("\nFirst 5 rows of monthly returns:")
display(rets_m.head())

mu_annual   = (1 + rets_m.mean())**12 - 1
Sigma_annual = rets_m.cov() * 12

print("\nAnnualized mean returns (preview):")
display(mu_annual.sort_values(ascending=False).to_frame("mu_annual"))

print("\nAnnualized covariance (preview):")
display(Sigma_annual)

Saved asset_returns.tsv with shape: (23, 7)

First 5 rows of monthly returns:


,AAPL,COST,MSFT,NVDA,SPY,TSLA
date,,,,,,
2023-10-31,-0.002570,-0.022161,0.070816,-0.062507,-0.021709,-0.197346
2023-11-30,0.113780,0.074918,0.122945,0.146886,0.091344,0.195379
2023-12-29,0.013583,0.138932,-0.007574,0.058934,0.045655,0.034988
2024-01-31,-0.042227,0.052721,0.057281,0.242418,0.015926,-0.246257
2024-02-29,-0.018543,0.072104,0.042318,0.285810,0.052187,0.077901



Annualized mean returns (preview):


,mu_annual
NVDA,1.217984
COST,0.359901
TSLA,0.349824
MSFT,0.318642
SPY,0.263275
AAPL,0.201881



Annualized covariance (preview):


,AAPL,COST,MSFT,NVDA,SPY,TSLA
AAPL,0.043521,0.012589,0.004669,0.011542,0.009987,0.039054
COST,0.012589,0.048661,0.012661,0.034777,0.016081,0.021049
MSFT,0.004669,0.012661,0.048533,0.058967,0.017853,0.026604
NVDA,0.011542,0.034777,0.058967,0.174271,0.034075,-0.004601
SPY,0.009987,0.016081,0.017853,0.034075,0.015508,0.029050
TSLA,0.039054,0.021049,0.026604,-0.004601,0.029050,0.320441


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [27]:
# YOUR CHANGES HERE

out = rets_m.copy().reset_index()
out = out.rename(columns={out.columns[0]: "date"})

out["date"] = pd.to_datetime(out["date"]).dt.strftime("%Y-%m-%d")

out.to_csv("historical_returns.tsv", sep="\t", index=False)

print("Saved historical_returns.tsv with shape:", out.shape)
display(out.head())

Saved historical_returns.tsv with shape: (23, 7)


,date,AAPL,COST,MSFT,NVDA,SPY,TSLA
0,2023-10-31,-0.002570,-0.022161,0.070816,-0.062507,-0.021709,-0.197346
1,2023-11-30,0.113780,0.074918,0.122945,0.146886,0.091344,0.195379
2,2023-12-29,0.013583,0.138932,-0.007574,0.058934,0.045655,0.034988
3,2024-01-31,-0.042227,0.052721,0.057281,0.242418,0.015926,-0.246257
4,2024-02-29,-0.018543,0.072104,0.042318,0.285810,0.052187,0.077901


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [28]:
# YOUR CHANGES HERE

rets = pd.read_csv("historical_returns.tsv", sep="\t")
rets["date"] = pd.to_datetime(rets["date"])
rets = rets.set_index("date")

mean_monthly_returns = rets.mean()

print("Estimated Expected Monthly Returns (mean of 23 months):")
display(mean_monthly_returns.to_frame("mean_monthly"))

mean_annual_returns = (1 + mean_monthly_returns)**12 - 1
print("\nAnnualized Expected Returns (for reference):")
display(mean_annual_returns.to_frame("mean_annual"))

Estimated Expected Monthly Returns (mean of 23 months):


,mean_monthly
AAPL,0.015442
COST,0.025949
MSFT,0.023318
NVDA,0.068636
SPY,0.019666
TSLA,0.025313



Annualized Expected Returns (for reference):


,mean_annual
AAPL,0.201881
COST,0.359901
MSFT,0.318642
NVDA,1.217984
SPY,0.263275
TSLA,0.349824


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [29]:
# YOUR CHANGES HERE

est = mean_monthly_returns.reset_index()
est.columns = ["asset", "estimated_return"]

est.to_csv("estimated_returns.tsv", sep="\t", index=False)

print("Saved estimated_returns.tsv with shape:", est.shape)
display(est)

Saved estimated_returns.tsv with shape: (6, 2)


,asset,estimated_return
0,AAPL,0.015442
1,COST,0.025949
2,MSFT,0.023318
3,NVDA,0.068636
4,SPY,0.019666
5,TSLA,0.025313


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [30]:
# YOUR CHANGES HERE

rets = pd.read_csv("historical_returns.tsv", sep="\t")
rets["date"] = pd.to_datetime(rets["date"])
rets = rets.set_index("date")

cov_matrix = rets.cov()

print("Covariance Matrix of Monthly Returns:")
display(cov_matrix)

cov_matrix_annual = cov_matrix * 12
print("\nAnnualized Covariance Matrix (Monthly × 12):")
display(cov_matrix_annual)

Covariance Matrix of Monthly Returns:


,AAPL,COST,MSFT,NVDA,SPY,TSLA
AAPL,0.003627,0.001049,0.000389,0.000962,0.000832,0.003254
COST,0.001049,0.004055,0.001055,0.002898,0.001340,0.001754
MSFT,0.000389,0.001055,0.004044,0.004914,0.001488,0.002217
NVDA,0.000962,0.002898,0.004914,0.014523,0.002840,-0.000383
SPY,0.000832,0.001340,0.001488,0.002840,0.001292,0.002421
TSLA,0.003254,0.001754,0.002217,-0.000383,0.002421,0.026703



Annualized Covariance Matrix (Monthly × 12):


,AAPL,COST,MSFT,NVDA,SPY,TSLA
AAPL,0.043521,0.012589,0.004669,0.011542,0.009987,0.039054
COST,0.012589,0.048661,0.012661,0.034777,0.016081,0.021049
MSFT,0.004669,0.012661,0.048533,0.058967,0.017853,0.026604
NVDA,0.011542,0.034777,0.058967,0.174271,0.034075,-0.004601
SPY,0.009987,0.016081,0.017853,0.034075,0.015508,0.029050
TSLA,0.039054,0.021049,0.026604,-0.004601,0.029050,0.320441


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [31]:
# YOUR CHANGES HERE

cov_matrix.to_csv("estimated_covariance.tsv", sep="\t")

print("Saved estimated_covariance.tsv with shape:", cov_matrix.shape)
display(cov_matrix)

Saved estimated_covariance.tsv with shape: (6, 6)


,AAPL,COST,MSFT,NVDA,SPY,TSLA
AAPL,0.003627,0.001049,0.000389,0.000962,0.000832,0.003254
COST,0.001049,0.004055,0.001055,0.002898,0.001340,0.001754
MSFT,0.000389,0.001055,0.004044,0.004914,0.001488,0.002217
NVDA,0.000962,0.002898,0.004914,0.014523,0.002840,-0.000383
SPY,0.000832,0.001340,0.001488,0.002840,0.001292,0.002421
TSLA,0.003254,0.001754,0.002217,-0.000383,0.002421,0.026703


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [32]:
# YOUR CHANGES HERE

est = pd.read_csv("estimated_returns.tsv", sep="\t")
cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

mu = est.set_index("asset")["estimated_return"].reindex(cov.columns)

best_asset = mu.idxmax()
weights = pd.Series(0.0, index=mu.index)
weights.loc[best_asset] = 1.0

mu_p_m = float(np.dot(weights.values, mu.values))
sigma_p_m = float(np.sqrt(weights.values @ cov.values @ weights.values))

mu_p_a = (1 + mu_p_m)**12 - 1
sigma_p_a = sigma_p_m * np.sqrt(12)

print("Maximum Return Portfolio (long-only, sum w = 1)")
print("------------------------------------------------")
print("Best asset by mean monthly return:", best_asset)
print("\nWeights:")
display(weights.to_frame("weight").style.format("{:.2%}"))

print("\nPortfolio Expected Return (monthly):  {:.2%}".format(mu_p_m))
print("Portfolio Volatility (monthly):        {:.2%}".format(sigma_p_m))
print("Portfolio Expected Return (annualized): {:.2%}".format(mu_p_a))
print("Portfolio Volatility (annualized):      {:.2%}".format(sigma_p_a))

Maximum Return Portfolio (long-only, sum w = 1)
------------------------------------------------
Best asset by mean monthly return: NVDA

Weights:


,weight
AAPL,0.00%
COST,0.00%
MSFT,0.00%
NVDA,100.00%
SPY,0.00%
TSLA,0.00%



Portfolio Expected Return (monthly):  6.86%
Portfolio Volatility (monthly):        12.05%
Portfolio Expected Return (annualized): 121.80%
Portfolio Volatility (annualized):      41.75%


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [33]:
# YOUR CHANGES HERE

if 'weights' not in globals():
    est = pd.read_csv("estimated_returns.tsv", sep="\t")
    mu = est.set_index("asset")["estimated_return"]
    best_asset = mu.idxmax()
    weights = pd.Series(0.0, index=mu.index)
    weights.loc[best_asset] = 1.0

df = weights.rename("allocation").reset_index()
df.columns = ["asset", "allocation"]

total = df["allocation"].sum()
if not np.isclose(total, 1.0):
    df["allocation"] = df["allocation"] / total

df.to_csv("maximum_return.tsv", sep="\t", index=False)

print("Saved maximum_return.tsv")
print("Sum of allocations:", df["allocation"].sum())
display(df)

Saved maximum_return.tsv
Sum of allocations: 1.0


,asset,allocation
0,AAPL,0.0
1,COST,0.0
2,MSFT,0.0
3,NVDA,1.0
4,SPY,0.0
5,TSLA,0.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [34]:
# YOUR CHANGES HERE

try:
    cov = cov_matrix.copy()
except NameError:
    cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

est = pd.read_csv("estimated_returns.tsv", sep="\t")
mu = est.set_index("asset")["estimated_return"].reindex(cov.columns)

n = len(cov)
ones = np.ones(n)

def gmv_long_only_slsqp(Sigma):
    from scipy.optimize import minimize

    def variance(w):
        w = np.asarray(w)
        return float(w @ Sigma @ w)

    cons = ({"type":"eq", "fun": lambda w: np.sum(w) - 1.0},)
    # bounds: 0 <= w_i <= 1
    bnds = tuple((0.0, 1.0) for _ in range(n))
    w0 = np.full(n, 1.0/n)

    res = minimize(variance, w0, method="SLSQP", bounds=bnds, constraints=cons)
    if not res.success:
        raise RuntimeError(f"SLSQP failed: {res.message}")
    return res.x

def gmv_unconstrained(Sigma):
    # Closed-form: w* ∝ Σ^{-1} 1, then normalize
    Sigma_inv = np.linalg.pinv(Sigma)
    w = Sigma_inv @ ones
    w = w / (ones @ w)
    return np.asarray(w)

try:
    w_gmv = gmv_long_only_slsqp(cov.values)
    method_used = "SLSQP (long-only)"
except Exception as e:
    w = gmv_unconstrained(cov.values)
    w = np.clip(w, 0, None)
    if w.sum() == 0:
        w = np.full(n, 1.0/n)
    w_gmv = w / w.sum()
    method_used = "Unconstrained closed-form → clipped & renormalized (approximate long-only)"

w_gmv = pd.Series(w_gmv, index=cov.columns, name="weight")

mu_p_m = float(w_gmv @ mu)
sig_p_m = float(np.sqrt(w_gmv.values @ cov.values @ w_gmv.values))
mu_p_a = (1 + mu_p_m)**12 - 1
sig_p_a = sig_p_m * np.sqrt(12)

print("Minimum Risk (GMV) Portfolio")
print(f"Method: {method_used}")
display(w_gmv.to_frame().style.format("{:.2%}"))

print("\nPortfolio Expected Return (monthly):  {:.2%}".format(mu_p_m))
print("Portfolio Volatility (monthly):        {:.2%}".format(sig_p_m))
print("Portfolio Expected Return (annualized): {:.2%}".format(mu_p_a))
print("Portfolio Volatility (annualized):      {:.2%}".format(sig_p_a))

Minimum Risk (GMV) Portfolio
Method: SLSQP (long-only)


,weight
AAPL,27.59%
COST,23.31%
MSFT,21.08%
NVDA,0.00%
SPY,28.01%
TSLA,0.00%



Portfolio Expected Return (monthly):  2.07%
Portfolio Volatility (monthly):        3.93%
Portfolio Expected Return (annualized): 27.93%
Portfolio Volatility (annualized):      13.60%


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [35]:
# YOUR CHANGES HERE

def _gmv_from_cov(cov_df):
    Sigma = cov_df.values
    n = Sigma.shape[0]
    try:
        from scipy.optimize import minimize
        def variance(w): return float(w @ Sigma @ w)
        cons = ({"type": "eq", "fun": lambda w: np.sum(w) - 1.0},)
        bnds = tuple((0.0, 1.0) for _ in range(n))
        w0 = np.full(n, 1.0 / n)
        res = minimize(variance, w0, method="SLSQP", bounds=bnds, constraints=cons)
        if not res.success:
            raise RuntimeError(res.message)
        return pd.Series(res.x, index=cov_df.columns, name="weight")
    except Exception:
        Sigma_inv = np.linalg.pinv(Sigma)
        w = Sigma_inv @ np.ones(n)
        w = np.clip(w / w.sum(), 0, None)
        if w.sum() == 0: w = np.full(n, 1.0 / n)
        return pd.Series(w / w.sum(), index=cov_df.columns, name="weight")

if 'w_gmv' in globals():
    weights_gmv = w_gmv.copy()
else:
    cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)
    weights_gmv = _gmv_from_cov(cov)

total = float(weights_gmv.sum())
if not np.isclose(total, 1.0):
    weights_gmv = weights_gmv / total

df = weights_gmv.rename("allocation").reset_index()
df.columns = ["asset", "allocation"]
df.to_csv("minimum_risk.tsv", sep="\t", index=False)

print("Saved minimum_risk.tsv")
print("Sum of allocations:", df["allocation"].sum())
display(df)

Saved minimum_risk.tsv
Sum of allocations: 1.0000000000000002


,asset,allocation
0,AAPL,2.759320e-01
1,COST,2.331440e-01
2,MSFT,2.108289e-01
3,NVDA,6.938894e-18
4,SPY,2.800951e-01
5,TSLA,8.550225e-18


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [36]:
# YOUR CHANGES HERE

from scipy.optimize import minimize

est = pd.read_csv("estimated_returns.tsv", sep="\t")
mu = est.set_index("asset")["estimated_return"]

cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)
cov = cov.reindex(index=mu.index, columns=mu.index)

Sigma = cov.values
mu_vec = mu.values
assets = mu.index.tolist()
n = len(assets)

def gmv_long_only(Sigma):
    n = Sigma.shape[0]
    def var_obj(w): return float(w @ Sigma @ w)
    cons = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]
    bnds = tuple((0.0, 1.0) for _ in range(n))
    w0 = np.full(n, 1.0/n)
    res = minimize(var_obj, w0, method="SLSQP", bounds=bnds, constraints=cons)
    if not res.success:
        raise RuntimeError(f"GMV solve failed: {res.message}")
    return res.x

try:
    w_gmv_np = w_gmv.values
except NameError:
    w_gmv_np = gmv_long_only(Sigma)
w_gmv = pd.Series(w_gmv_np, index=assets)

best_asset = mu.idxmax()
w_max = pd.Series(0.0, index=assets)
w_max.loc[best_asset] = 1.0

r_gmv = float(w_gmv @ mu)
r_max = float(w_max @ mu)

targets = np.linspace(r_gmv, r_max, 101)

def min_var_for_target(Sigma, mu_vec, target, w_start=None):
    n = len(mu_vec)
    if w_start is None:
        w_start = np.full(n, 1.0/n)
    def var_obj(w): return float(w @ Sigma @ w)
    cons = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type": "eq", "fun": lambda w, tr=target: float(w @ mu_vec - tr)},
    ]
    bnds = tuple((0.0, 1.0) for _ in range(n))
    res = minimize(var_obj, w_start, method="SLSQP", bounds=bnds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})
    return res

frontier_weights = []
achieved = []
vols = []

w_prev = w_gmv_np.copy()
for tr in targets:
    res = min_var_for_target(Sigma, mu_vec, tr, w_start=w_prev)
    if not res.success:
        r_prev = float(w_prev @ mu_vec)
        r_best = float(w_max.values @ mu_vec)
        if r_best == r_prev:
            w_sol = w_prev
        else:
            alpha = np.clip((tr - r_prev) / (r_best - r_prev), 0.0, 1.0)
            w_sol = (1 - alpha) * w_prev + alpha * w_max.values
    else:
        w_sol = res.x

    w_prev = w_sol
    frontier_weights.append(w_sol)
    achieved.append(float(w_sol @ mu_vec))
    vols.append(float(np.sqrt(w_sol @ Sigma @ w_sol)))

frontier_weights[0]  = w_gmv_np
achieved[0]          = r_gmv
vols[0]              = float(np.sqrt(w_gmv_np @ Sigma @ w_gmv_np))
frontier_weights[-1] = w_max.values
achieved[-1]         = r_max
vols[-1]             = float(np.sqrt(w_max.values @ Sigma @ w_max.values))

frontier = pd.DataFrame({
    "target_return": targets,
    "achieved_return": achieved,
    "volatility": vols
})
print("Efficient frontier computed (monthly units). Shape:", frontier.shape)

display(frontier.head(3))
display(frontier.iloc[[50]])
display(frontier.tail(3))


Efficient frontier computed (monthly units). Shape: (101, 3)


,target_return,achieved_return,volatility
0,0.020735,0.020735,0.039256
1,0.021214,0.021214,0.036774
2,0.021693,0.021693,0.037244


,target_return,achieved_return,volatility
50,0.044686,0.044686,0.071228


,target_return,achieved_return,volatility
98,0.067678,0.067678,0.117830
99,0.068157,0.068157,0.119156
100,0.068636,0.068636,0.120510


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [37]:
# YOUR CHANGES HERE

assert 'frontier_weights' in globals(), "frontier_weights not found — run the frontier construction cell first."
assets = list(mu.index)

W = pd.DataFrame(frontier_weights, columns=assets)

row_sums = W.sum(axis=1).replace(0, np.nan)
W = W.div(row_sums, axis=0)

port_return = W.values @ mu.values
port_risk = np.sqrt(np.einsum('ij,jk,ik->i', W.values, cov.values, W.values))

out = pd.DataFrame({
    "index": np.arange(len(W), dtype=int),
    "return": port_return,
    "risk": port_risk,
})
out = pd.concat([out, W], axis=1)

out.to_csv("efficient_frontier.tsv", sep="\t", index=False)

print("Saved efficient_frontier.tsv with shape:", out.shape)
display(out.head(3))
display(out.tail(3))

Saved efficient_frontier.tsv with shape: (101, 9)


,index,return,risk,AAPL,COST,MSFT,NVDA,SPY,TSLA
0,0,0.020735,0.039256,0.275932,0.233144,0.210829,6.938894e-18,0.280095,8.550225e-18
1,1,0.021214,0.036774,0.106875,0.058576,0.000000,3.331279e-02,0.801236,0.000000e+00
2,2,0.021693,0.037244,0.108261,0.062914,0.000000,4.265755e-02,0.786168,5.399112e-19


,index,return,risk,AAPL,COST,MSFT,NVDA,SPY,TSLA
98,98,0.067678,0.117830,0.000000e+00,4.470399e-19,5.909943e-19,0.977887,0.0,0.022113
99,99,0.068157,0.119156,4.040315e-18,0.000000e+00,0.000000e+00,0.988943,0.0,0.011057
100,100,0.068636,0.120510,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.0,0.000000


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [38]:
# YOUR CHANGES HERE

mu  = pd.read_csv("estimated_returns.tsv", sep="\t").set_index("asset")["estimated_return"]
cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0).reindex(index=mu.index, columns=mu.index)

assets = mu.index.tolist()
n = len(assets)

Sigma = cov.values
Sigma = (Sigma + Sigma.T) / 2.0
eigvals = np.linalg.eigvalsh(Sigma)
if eigvals.min() <= 0:
    Sigma = Sigma + np.eye(n) * (1e-12 - eigvals.min() + 1e-12)

T = 23
B = 1000
np.random.seed(42)

winners_idx = np.empty(B, dtype=int)
for b in range(B):
    sample = np.random.multivariate_normal(mean=mu.values, cov=Sigma, size=T)

    mu_hat = sample.mean(axis=0)

    winners_idx[b] = int(np.argmax(mu_hat))

winners_assets = pd.Series(winners_idx).map(dict(enumerate(assets)))
counts = winners_assets.value_counts().reindex(assets).fillna(0).astype(int)
shares = counts / B

res = pd.DataFrame({"wins": counts, "share": shares}).sort_values("share", ascending=False)

best_asset = mu.idxmax()
print("Original max-return asset:", best_asset)
print(f"Share of resamples where {best_asset} remains best: {res.loc[best_asset, 'share']:.2%}\n")

display(res)

Original max-return asset: NVDA
Share of resamples where NVDA remains best: 83.60%



,wins,share
NVDA,836,0.836
TSLA,140,0.140
COST,15,0.015
AAPL,9,0.009
MSFT,0,0.000
SPY,0,0.000


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [39]:
# YOUR CHANGES HERE

mu  = pd.read_csv("estimated_returns.tsv", sep="\t").set_index("asset")["estimated_return"]
cov = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0).reindex(index=mu.index, columns=mu.index)

if 'res' in globals() and 'share' in getattr(res, 'columns', []) and set(res.index) == set(mu.index):
    probs = res['share'].reindex(mu.index).fillna(0.0)
else:
    Sigma = cov.values
    Sigma = (Sigma + Sigma.T) / 2.0
    eigvals = np.linalg.eigvalsh(Sigma)
    if eigvals.min() <= 0:
        Sigma = Sigma + np.eye(len(mu)) * (1e-12 - eigvals.min() + 1e-12)

    T = 23
    B = 1000
    np.random.seed(42)

    winners_idx = np.empty(B, dtype=int)
    for b in range(B):
        sample = np.random.multivariate_normal(mean=mu.values, cov=Sigma, size=T)
        mu_hat = sample.mean(axis=0)
        winners_idx[b] = int(np.argmax(mu_hat))

    counts = pd.Series(winners_idx).map(dict(enumerate(mu.index))).value_counts().reindex(mu.index).fillna(0).astype(int)
    probs = (counts / B).reindex(mu.index)

out = probs.to_frame("probability").reset_index()
if out.columns[0] != "asset":
    out = out.rename(columns={out.columns[0]: "asset"})

out.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

print("Saved max_return_probabilities.tsv with shape:", out.shape)
print("Sum of probabilities:", float(out['probability'].sum()))
display(out)

Saved max_return_probabilities.tsv with shape: (6, 2)
Sum of probabilities: 1.0


,asset,probability
0,AAPL,0.009
1,COST,0.015
2,MSFT,0.000
3,NVDA,0.836
4,SPY,0.000
5,TSLA,0.140


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [41]:
from datetime import date

ack_text = f"""DX704 Week 1 — Acknowledgments
Date: {date.today().isoformat()}

People / Discussions
- None.

External Libraries (beyond standard course stack)
- yfinance — downloaded adjusted close prices.
- scipy.optimize (SLSQP) — solved constrained mean–variance problems.

Data Sources
- Yahoo Finance (via yfinance).

Generative AI Usage
- None.

Other Sources
- DX601–DX704 example notebooks referenced as allowed.
"""

with open("acknowledgements.txt", "w", encoding="utf-8") as f:
    f.write(ack_text)

import os
print("Exists?", os.path.exists("acknowledgements.txt"), "Size:", os.path.getsize("acknowledgements.txt"), "bytes")

Exists? True Size: 401 bytes


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

In [43]:
rets = pd.read_csv("historical_returns.tsv", sep="\t")
rets["date"] = pd.to_datetime(rets["date"])
rets = rets.set_index("date").sort_index()

assets = rets.columns.tolist()
n = len(assets)

mu = rets.mean()
mu_df = mu.rename("estimated_return").rename_axis("asset").reset_index()
mu_df.to_csv("estimated_returns.tsv", sep="\t", index=False)

cov = rets.cov()
cov.to_csv("estimated_covariance.tsv", sep="\t")

mu_vec = mu.values
Sigma  = cov.values

def gmv_long_only(Sigma: np.ndarray) -> np.ndarray:
    n = Sigma.shape[0]
    def var_obj(w): return float(w @ Sigma @ w)
    cons = ({"type":"eq", "fun": lambda w: np.sum(w) - 1.0},)
    bnds = tuple((0.0, 1.0) for _ in range(n))
    w0 = np.full(n, 1.0/n)
    res = minimize(var_obj, w0, method="SLSQP", bounds=bnds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})
    if not res.success:
        raise RuntimeError(f"GMV solve failed: {res.message}")
    return res.x

def min_var_for_target(Sigma: np.ndarray, mu_vec: np.ndarray, target: float, w_start=None) -> np.ndarray:
    n = len(mu_vec)
    if w_start is None:
        w_start = np.full(n, 1.0/n)
    def var_obj(w): return float(w @ Sigma @ w)
    cons = (
        {"type":"eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type":"eq", "fun": lambda w, tr=target: float(w @ mu_vec - tr)},
    )
    bnds = tuple((0.0, 1.0) for _ in range(n))
    res = minimize(var_obj, w_start, method="SLSQP", bounds=bnds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})
    if not res.success:
        raise RuntimeError(f"Target-return solve failed: {res.message}")
    return res.x

best_idx = int(np.argmax(mu_vec))
w_max = np.zeros(n); w_max[best_idx] = 1.0
pd.DataFrame({"asset": assets, "allocation": w_max}).to_csv("maximum_return.tsv", sep="\t", index=False)

w_gmv = gmv_long_only(Sigma)
pd.DataFrame({"asset": assets, "allocation": w_gmv}).to_csv("minimum_risk.tsv", sep="\t", index=False)

r_gmv = float(w_gmv @ mu_vec)
r_max = float(w_max @ mu_vec)
targets = np.linspace(r_gmv, r_max, 101)

frontier_weights = []
achieved = []
vols = []
w_prev = w_gmv.copy()
for tr in targets:
    try:
        w_sol = min_var_for_target(Sigma, mu_vec, tr, w_start=w_prev)
    except RuntimeError:
        r_prev = float(w_prev @ mu_vec)
        if r_max == r_prev:
            w_sol = w_prev
        else:
            alpha = np.clip((tr - r_prev) / (r_max - r_prev), 0.0, 1.0)
            w_sol = (1 - alpha) * w_prev + alpha * w_max
            w_sol = np.clip(w_sol, 0, 1)
            s = w_sol.sum()
            w_sol = w_sol / s if s > 0 else np.full(n, 1.0/n)
    w_prev = w_sol
    frontier_weights.append(w_sol)
    achieved.append(float(w_sol @ mu_vec))
    vols.append(float(np.sqrt(w_sol @ Sigma @ w_sol)))

frontier_weights[0]  = w_gmv
achieved[0]          = r_gmv
vols[0]              = float(np.sqrt(w_gmv @ Sigma @ w_gmv))
frontier_weights[-1] = w_max
achieved[-1]         = r_max
vols[-1]             = float(np.sqrt(w_max @ Sigma @ w_max))

W = pd.DataFrame(frontier_weights, columns=assets)
row_sums = W.sum(axis=1).replace(0, np.nan)
W = W.div(row_sums, axis=0)

front = pd.DataFrame({
    "index": np.arange(101, dtype=int),
    "return": achieved,
    "risk": vols
})
front = pd.concat([front, W], axis=1)
front.to_csv("efficient_frontier.tsv", sep="\t", index=False)

Sigma_psd = (Sigma + Sigma.T) / 2.0
eigvals = np.linalg.eigvalsh(Sigma_psd)
if eigvals.min() <= 0:
    Sigma_psd = Sigma_psd + np.eye(n) * (1e-12 - eigvals.min() + 1e-12)

T = 23
B = 1000
np.random.seed(42)

winners = np.empty(B, dtype=int)
for b in range(B):
    sample = np.random.multivariate_normal(mean=mu_vec, cov=Sigma_psd, size=T)
    mu_hat = sample.mean(axis=0)
    winners[b] = int(np.argmax(mu_hat))

counts = pd.Series(winners).value_counts().reindex(range(n)).fillna(0).astype(int).values
probs  = counts / B
probs_df = pd.DataFrame({"asset": assets, "probability": probs})
probs_df.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

print("Recreated files:")
for fn in [
    "estimated_returns.tsv",
    "estimated_covariance.tsv",
    "maximum_return.tsv",
    "minimum_risk.tsv",
    "efficient_frontier.tsv",
    "max_return_probabilities.tsv",
]:
    print(" -", fn)

print("\nFrontier shape:", front.shape)
print("Sum of each frontier weight row (min/max):",
      float(W.sum(axis=1).min()), float(W.sum(axis=1).max()))

Recreated files:
 - estimated_returns.tsv
 - estimated_covariance.tsv
 - maximum_return.tsv
 - minimum_risk.tsv
 - efficient_frontier.tsv
 - max_return_probabilities.tsv

Frontier shape: (101, 9)
Sum of each frontier weight row (min/max): 0.9999999999999998 1.0000000000000002


Submit "project.ipynb" in Gradescope.